In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # import geocoder
import requests 
from bs4 import BeautifulSoup 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [23]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


In [24]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected.')

Data Collected.


In [25]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood


In [26]:
for data in range(len(neighborhoods)):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name}, ignore_index=True)

neighbors

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,M2A\n
1,M3A\n,North York\n,Parkwoods\n
2,M4A\n,North York\n,Victoria Village\n
3,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
4,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n"
...,...,...,...
130,M9Y\n,Not assigned\n,M1Z\n
131,M2Z\n,Not assigned\n,M3Z\n
132,M4Z\n,Not assigned\n,M5Z\n
133,M6Z\n,Not assigned\n,M7Z\n


In [27]:
neighbors.shape

(135, 3)

In [28]:
neighbors.drop(neighbors[neighbors['Borough']=="Not assigned\n"].index,axis=0, inplace=True)
neighbors=neighbors.groupby("PostalCode").agg(lambda x:','.join(set(x)))

# If a cell has a borough but a Not assigned neighborhood, 
# then the neighborhood will be the same as the borough. 
# So for the 9th cell in the table on the Wikipedia page, 
# the value of the Borough and the Neighborhood columns will be Queen's Park.

neighbors.loc[neighbors['Neighborhood']=="Not assigned\n",'Neighborhood']=neighbors.loc[neighbors['Neighborhood']=="Not assigned\n",'Borough']

neighbors.shape

(91, 2)

In [29]:
neighbors.head(50)

,Borough,Neighborhood
PostalCode,,
M1B\n,Scarborough\n,"Malvern, Rouge\n"
M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek\n"
M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill\n"
M1M\n,Scarborough\n,"Cliffside, Cliffcrest, Scarborough Village West\n"
M1N\n,Scarborough\n,"Birch Cliff, Cliffside West\n"
M1P\n,Scarborough\n,"Dorset Park, Wexford Heights, Scarborough Town..."
M1R\n,Scarborough\n,"Wexford, Maryvale\n"
M1S\n,Scarborough\n,Agincourt\n
M1V\n,Scarborough\n,"Milliken, Agincourt North, Steeles East, L'Amo..."
